In [1]:
import pandas as pd

df = pd.DataFrame(data=[['2022-01-01'], ['2023-01-19']], columns=['date'])

In [14]:
df['date'] = pd.to_datetime(df['date'])

In [17]:
df['date'].dt.tz_localize(None)

0   2022-01-01
1   2023-01-19
Name: date, dtype: datetime64[ns]